### Imports

In [ ]:
%pip install openai --upgrade
%pip install pandas

In [ ]:
import os
import csv
import openai
import pandas as pd

### OPENAI API Keys

In [ ]:
openai.api_key = 'PLEASE_GET_YOUR_KEY_AND_MAKE_SURE_YOU_HAVE_SAVINGS_IN_OPENAI'

### Loading Dataset

In [44]:
df = pd.read_csv("the_predict_result.csv", delimiter=',', index_col=False, encoding='latin1')
df.loc[ df["label"] == 'positive', "label"] = 2 # Positive classification
df.loc[ df["label"] == 'neutral', "label"] = 1 # Neutral classification
df.loc[ df["label"] == 'negative', "label"] = 0 # Negative classification

In [45]:
text = df.text.values
truelabel = df.truth.values
label = df.label.values

In [47]:
print("There are ", len(df[df["label"]==2]) , "positive examples in this dataset.")
print("There are ", len(df[df["label"]==1]), "neutral examples in this dataset.")
print("There are ", len(df[df["label"]==0]), "negative examples in this dataset.")

There are  76 positive examples in this dataset.
There are  262 neutral examples in this dataset.
There are  33 negative examples in this dataset.


In [ ]:
def find_longest_string_length(lst):
    longest_str = max(lst, key=len)
    return len(longest_str)

result = find_longest_string_length(text)
print("The longest string length of this dataset is:", result)

In [ ]:
def count_words_in_longest_string(lst):
    longest_str = max(lst, key=len)
    words_in_longest_str = len(longest_str.split())
    return words_in_longest_str

result = count_words_in_longest_string(text)
print("The longest text length is:", result)


In [ ]:
def average_word_count(lst):
    total_words = 0
    total_strings = len(lst)
    for string in lst:
        words_in_string = len(string.split())
        total_words += words_in_string

    average_words = total_words / total_strings
    return average_words

result = average_word_count(text)
print("The average word number is:", result)

In [ ]:
# you can check the performance of the BERT prediction firstly
from sklearn.metrics import classification_report
print(classification_report(list(truelabel), list(label), labels=[0,1,2]))

#### Constraint

In [36]:
testtext = []
testid1 = []
for i, v in enumerate(label):
    if v == 0:
        testtext.append(text[i])
        testid1.append(testid[i])

### Prompt

In [ ]:
# This is the prompt for Task 2 only!
prompt = '''
You are given a tweet which is classified to be tweet that has negative sentiment.
Your job is to take the label in the end and identify whether it is true or false according to the following rules:
1. If you find the input tweets truly contain negative sentiment, return True, 
2. If you find the input tweets contains positive or neutral sentiment, return false.

Here are some examples: 
Input: I'm so sad to eat the drug that I want to kill my self, Label: 1
Output: False, because the user explicitly said he has negative sentiment. But the given label is 1.
Input I am so angry with my therapy and I do not want to eat the drug anymore. Label 0
Output: True, because the user explicitly express his negative sentiment acorss medicine.

Your output should only be True plus one sentence of your reason which point out which sentence in the text can be a proof step by step, your output should within 30 words if you think the tweet truly contain negative sentiment,
or False and your re-classification result (positive or neutral) plus one sentence of your reason which point out which sentence in the text can be a proof step by step, your output should within 30 words  if you think the tweet do not contain negative sentiment.
You should return True for most uncertain cases and be careful to return False.
'''
print(prompt)

In [15]:
prompts = testtext
id1 = testid1    

In [17]:
# preparing a message for chat models
def prepare_message(uinput):
    role="user"
    line = {"role": role, "content": uinput}
    
    assert1={"role": "system", "content": "You are a highly intelligent and accurate therapy sentiment classification explainable system."}
    assert2={"role": "assistant", "content": prompt}
    assert3=line
    iprompt = []
    iprompt.append(assert1)
    iprompt.append(assert2)
    iprompt.append(assert3)
    
    return iprompt

### GPT4 Inference if you have

In [18]:
# # we can use gpt3.5
# uinput="Is the following tweet postive or negative:She loved the new Batman movie! But I agreed with that critic we just heard."
# iprompt=prepare_message(uinput) #preparing the messages for ChatGPT
# response=openai.ChatCompletion.create(model="gpt-3.5-turbo",messages=iprompt) #ChatGPT dialog
# text=response["choices"][0]["message"]["content"] #response in JSON
# print("ChatGPT response:",text)

In [19]:
# # I have applied for gpt4 but have not got the permission
# uinput=inputs
# iprompt=prepare_message(uinput) #preparing the messages for ChatGPT
# response=openai.ChatCompletion.create(model="gpt-4",messages=iprompt) #ChatGPT dialog
# text=response["choices"][0]["message"]["content"] #response in JSON
# print("ChatGPT response:",text)

### GPT3.5 inference pipeline

In [20]:
def gpt(uinput):
    iprompt=prepare_message(uinput) #preparing the messages for ChatGPT
    response=openai.ChatCompletion.create(model="gpt-3.5-turbo",messages=iprompt) #ChatGPT dialog
    text=response["choices"][0]["message"]["content"] #response in JSON
    print("ChatGPT response:",text)
    return text

In [24]:
import time
predicts = []
for t, i in enumerate(prompts):
    if (t + 1) % 3 == 0:
        time.sleep(60)  # 停顿一分钟
    
    print(t+ 1,':', testid1[t])
    inputs = i
    predict = str(gpt(inputs))
    predicts.append(predict)

1 : 1.53e+18
ChatGPT response: False. The tweet does not contain negative sentiment. The user mentions quitting Adderall without much issue and considers themselves lucky. The positive sentiment can be seen in the statement "I'm def lucky."
2 : 1.5e+18
ChatGPT response: True. The tweet contains negative sentiment because the user expresses dissatisfaction and describes their experience as "hell."
3 : 1.51e+18
ChatGPT response: True. The tweet expresses dissatisfaction with antidepressants and mood stabilizers, indicating negative sentiment towards therapy.
4 : 1.49e+18
ChatGPT response: True. The sentence "i shouldn't have taken two adderall before leaving im quite literally vibrating" expresses regret and physical discomfort, indicating a negative sentiment.
5 : 1.56e+18
ChatGPT response: False. The sentiment in the tweet is classified as negative, but it is actually neutral. The user expresses a desire to convince their mom to switch medications, which does not necessarily indicate a

### Correction

In [ ]:
correction = []
for i , v in enumerate(predicts):
    if "false" in v or "False" in v or "FALSE" in v:
        correction.append(testid1[i])

In [ ]:
for i , v in enumerate(testid1):
    if v in correction:
        print('correct id', v)
        label[i] = 0

In [ ]:
# Compare the performance
print(classification_report(list(truelabel), list(label), labels=[0,1,2]))

#### Saving the results

In [26]:
import pandas as pd
def save_lists_to_csv(list1, list2, filename):
    data_dict = {
        'id': list1,
        'bool': [item.split()[0] for item in list2],
        'explanation': [' '.join(item.split()[1:]) for item in list2]
    }

    df = pd.DataFrame(data_dict)
    df.to_csv(filename, index=False)

csv_filename = 'ALEX_task2_label0_corrected.csv'
save_lists_to_csv(testid1, predicts, csv_filename)